# RIOT's API: Obtención y guardado de datos

In [3]:
!pip install tqdm

In [4]:
import os

OUTPUT_PATH = "data"
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Cambia la ruta a la carpeta donde está tu script en Google Drive
script_dir = '/content/drive/MyDrive/PPS_Esports'

# Cambiar al directorio del script
os.chdir(script_dir)

# Crear la carpeta 'data' si no existe
DATA_DIR = os.path.join(script_dir, 'data')
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

Mounted at /content/drive


## Definición de funciones útiles

COMPLETAR la siguiente variable con el valor de tu API KEY, se saca de la pagina de [RIOT](https://developer.riotgames.com/)

In [6]:
API_KEY = "RGAPI-d311affd-5ae7-445d-932e-9443d9d66103"  # REEMPLAZAR

Es limitante la cantidad de consultas que permite una PERSONAL APIKEY:
- 20 requests cada 1 segundo
- 100 requests cada 2 minutos
Si queremos un limite mas amplio hay que intentar conseguir una PRODUCTION  APIKEY. Esta requiere un proceso de solicitud y respuesta.

El siguiente código especifica un serie de funciones que permiten la obtención de datos diversos de partidas de LOL.

In [7]:
import requests
import random

#URL_LOL = "https://la2.api.riotgames.com"
URL_LOL = "https://na1.api.riotgames.com"
URL_RIOT = "https://americas.api.riotgames.com"


# def get_challenges_config(api_key):
#     """
#     Obtiene una lista de la configuración básica de desafíos.

#     Parameters:
#       api_key (str): La clave de API de Riot para autenticar la solicitud.

#     Returns:
#       list of dict: Una lista de diccionarios, donde cada diccionario representa un desafío y contiene los siguientes campos:
#         - id (int): Identificador único del desafío.
#         - localizedNames (dict): Diccionario con la información de nombres traducidos del desafío. Las claves son los códigos de idioma (por ejemplo, "ar_AE", "cs_CZ", etc.), y los valores son diccionarios con:
#           - description (str): Descripción del desafío en el idioma correspondiente.
#           - name (str): Nombre del desafío en el idioma correspondiente.
#           - shortDescription (str): Descripción corta del desafío en el idioma correspondiente.
#     """
#     endpoint = f"{URL_LOL}/lol/cha
#     else:
#         response.raise_for_status()


def get_challenge_leaderboard(api_key, challenge_id, level):
    """
    Obtiene el tablero de líderes de un desafío específico para un nivel determinado.

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      challenge_id (int): El identificador único del desafío para el cual se desea obtener el tablero de líderes.
      level (str): El nivel del desafío. Debe ser uno de los siguientes: "MASTER", "GRANDMASTER" o "CHALLENGER".

    Returns:
      list of dict: Una lista de diccionarios, donde cada diccionario representa a un jugador en el tablero de líderes y contiene los siguientes campos:
        - position (int): La posición del jugador en el tablero de líderes.
        - puuid (str): Identificador único del jugador.
        - value (int): El valor asociado al desafío para el jugador en la posición dada.
    """
    endpoint = f"{URL_LOL}/lol/challenges/v1/challenges/{challenge_id}/leaderboards/by-level/{level}"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}

    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


def get_account_by_puuid(api_key, puuid):
    """
    Obtiene la información de una cuenta a partir del identificador único del jugador (PUUID, Player Universally Unique IDentifiers).

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      puuid (str): El identificador único del jugador para el cual se desea obtener la información de la cuenta.

    Returns:
      dict: Un diccionario que representa la cuenta del jugador con los siguientes campos:
        - puuid (str): El identificador único del jugador.
        - gameName (str): El nombre del jugador en el juego.
        - tagLine (str): La etiqueta del jugador, que suele representar la región o el servidor.
    """
    endpoint = f"{URL_RIOT}/riot/account/v1/accounts/by-puuid/{puuid}"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}

    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


def get_match_ids(api_key, puuid, type, start=0, count=20):
    """
    Obtiene una lista de identificadores de partidas para un jugador específico.

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      puuid (str): El identificador único del jugador para el cual se desean obtener los identificadores de partidas.
      type (str): El tipo de partidas a recuperar. Puede ser uno de los siguientes: "ranked", "normal", "tourney", "tutorial".
      start (int, opcional): El índice de la primera partida a recuperar en la lista. El valor por defecto es 0.
      count (int, opcional): El número de partidas a recuperar, comenzando desde el índice `start`. El valor por defecto es 20.

    Returns:
      list of str: Una lista de identificadores de partidas en formato de cadena. Los identificadores representan las partidas que cumplen con los parámetros especificados.
    """
    endpoint = f"{URL_RIOT}/lol/match/v5/matches/by-puuid/{puuid}/ids"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}
    params = {"type": type, "start": start, "count": count, "api_key": api_key}

    response = requests.get(endpoint, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


def get_match_details(api_key, match_id):
    """
    Obtiene la información detallada de una partida específica.

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      match_id (str): El identificador único de la partida para la cual se desea obtener los detalles.

    Returns:
      dict: Un diccionario con la información detallada sobre la partida especificada.

      Para más detalles sobre la estructura y los campos de la respuesta, consulta la documentación oficial de la API en la URL: https://developer.riotgames.com/apis#match-v5/GET_getMatch.
    """
    endpoint = f"{URL_RIOT}/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}

    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()


def get_match_timeline(api_key, match_id):
    """
    Obtiene la línea de tiempo detallada de una partida específica.

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      match_id (str): El identificador único de la partida para la cual se desea obtener la línea de tiempo.

    Returns:
      dict: Un diccionario con la información detallada sobre la línea de tiempo de la partida especificada.

      Para más detalles sobre la estructura de la respuesta, consulta la documentación oficial de la API en la URL: https://developer.riotgames.com/apis#match-v5/GET_getTimeline.
    """
    endpoint = f"{URL_RIOT}/lol/match/v5/matches/{match_id}/timeline"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}

    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Alternativa:

Lo que planeabamos desde un inicio, recuperar un listado ordenado de los mejores jugadores de cierto conjunto de ligas (Master, Grandmaster, y Challenger). En cierto servidor (ej: LAS, LAN, EU, etc).

+1000 jugadores.
20 partidas por jugador.

En el mejor de los casos --> `1000 * 20 = 20.000` partidos.

CODIGO DE AYUDA: https://github.com/razieloren/leaguetracker/blob/main/riot/modules/consts.py

 ## Objetos y funciones útiles

In [8]:
from enum import Enum
from collections import namedtuple

RegionProps = namedtuple("RegionProps", "endpoint friendly_name")


class Region(Enum):
    BRAZIL = RegionProps("br1", "Brazil")
    EUNE = RegionProps("eun1", "Europe Nordic & East")
    EUW = RegionProps("euw1", "Europe West")
    JP = RegionProps("jp1", "Japan")
    KR = RegionProps("kr", "Korea")
    LATIN1 = RegionProps("la1", "Latin America North")
    LATIN2 = RegionProps("la2", "Latin America South")
    NA = RegionProps("na1", "North America")
    OC = RegionProps("oc1", "Oceania")
    TR = RegionProps("tr1", "Turkey")
    RU = RegionProps("ru", "Russia")
    PH = RegionProps("ph2", "The Philippines")
    SG = RegionProps("sg2", "Singapore, Malaysia, & Indonesia")
    TH = RegionProps("th2", "Thailand")
    TW = RegionProps("tw2", "Taiwan, Hong Kong, and Macao")
    VN = RegionProps("vn2", "Vietnam")


class Product(Enum):
    LeagueOfLegends = "lol"


class Queue(Enum):
    RankedFlex = "RANKED_FLEX_SR"
    RankedSolo = "RANKED_SOLO_5x5"

In [9]:
import requests
import random


class API:

    _BASE_URL = "https://{region}.api.riotgames.com"

    def __init__(self, region: Region, api_key: str):
        self._region = region
        self._api_key = api_key

    @property
    def base_url(self):
        return self._BASE_URL.format(region=str(self._region.value.endpoint))

    def get_challengers_top(self, queque: Queue):
        """
        DOC: https://developer.riotgames.com/apis#league-v4/GET_getChallengerLeague
        LEAGUE-v4
        Devuelve un arreglo con informacion ordenada de los mejores jugadores
        challenger dado:
        - queue: ejemplo RANKED_SOLO_5x5
        """

        url = f"{self.base_url}/lol/league/v4/challengerleagues/by-queue/{queque.value}"
        print(url)
        headers = {"X-Riot-Token": self._api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    def get_grandmaster_top(self, queque: Queue):
        """
        DOC: https://developer.riotgames.com/apis#league-v4/GET_getChallengerLeague
        LEAGUE-v4
        Devuelve un arreglo con informacion ordenada de los mejores jugadores
        grandmaster dado:
        - queue: ejemplo RANKED_SOLO_5x5
        """

        url = (
            f"{self.base_url}/lol/league/v4/grandmasterleagues/by-queue/{queque.value}"
        )
        print(url)
        headers = {"X-Riot-Token": self._api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    def get_master_top(self, queque: Queue):
        """
        DOC: https://developer.riotgames.com/apis#league-v4/GET_getChallengerLeague
        LEAGUE-v4
        Devuelve un arreglo con informacion ordenada de los mejores jugadores
        master dado:
        - queue: ejemplo RANKED_SOLO_5x5
        """

        url = f"{self.base_url}/lol/league/v4/masterleagues/by-queue/{queque.value}"
        print(url)
        headers = {"X-Riot-Token": self._api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

In [10]:
# Clase para abtraer la logica de tiempos de espera para la api
import time

class developSleepLogic:
    _perSecond = 20
    _perMinute = 100

    def __init__(self):
        self.listado_consultas = []

    def count(self):
        remaining_time = 0
        current_time = time.time()
        self.listado_consultas.append(current_time)
        last_second_consultas = [consulta for consulta in self.listado_consultas if current_time - consulta <= 1]
        last_minute_consultas = [consulta for consulta in self.listado_consultas if current_time - consulta <= 60]
        if (len(last_second_consultas) >= self._perSecond) and (len(last_minute_consultas)<self._perMinute):
            elapsed_time = current_time - last_minute_consultas[0]
            remaining_time = 1 - elapsed_time
            return

        if (len(last_second_consultas) >= self._perMinute):
            elapsed_time = current_time - last_minute_consultas[0]
            remaining_time = 120 - elapsed_time

        self.listado_consultas = last_minute_consultas

        # Dormir
        if (remaining_time > 0):
          time.sleep(remaining_time)

 ## Recuperar listado json de jugadores Challenger, GrandMaster y Master

In [ ]:
# Recuperar ids de todos los jugadores de Challenger, GrandMaster y Master
import json

# API_KEY = "RGAPI-0b377e8d-004a-4f7d-95be-7b6ce88bbead"  # Actualizar API KEY cada día

api = API(Region.NA, API_KEY)

data_challenger = api.get_challengers_top(Queue.RankedSolo)
with open(f"{OUTPUT_PATH}/get_challengers_top.json", "w") as file:
    json.dump(data_challenger, file, indent=4)

data_grandmaster = api.get_grandmaster_top(Queue.RankedSolo)
with open(f"{OUTPUT_PATH}/get_grandmaster_top.json", "w") as file:
    json.dump(data_grandmaster, file, indent=4)

data_master = api.get_master_top(Queue.RankedSolo)
with open(f"{OUTPUT_PATH}/get_master_top.json", "w") as file:
    json.dump(data_master, file, indent=4)

https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
https://na1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5
https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5


In [ ]:
print(len(data_challenger["entries"]))
print(len(data_grandmaster["entries"]))
print(len(data_master["entries"]))

88
411
1633


In [15]:
# "summonerId" -> "puuid"
def get_puuid_by_summoner_id(api_key, summoner_id):
    """
    Obtiene la información de un summoner a partir del ID del summoner.

    Parameters:
      api_key (str): La clave de API de Riot para autenticar la solicitud.
      summoner_id (str): El identificador único del summoner para el cual se desea obtener la información.

    Returns:
      dict: Un diccionario que representa el summoner con los siguientes campos:
        - id (str): El identificador único del summoner.
        - puuid (str): El identificador PUUID asociado al summoner.
        - accountId (str): El identificador de cuenta asociado al summoner.
        - profileIconId (int): ID del icono de perfil asociado al summoner.
        - summonerLevel (int): Nivel del summoner asociado con el summoner.
    """
    endpoint = f"{URL_LOL}/lol/summoner/v4/summoners/{summoner_id}"
    headers = {"X-Riot-Token": api_key, "User-agent": ""}

    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [ ]:
# Leer los Jsons 'get_challengers_top.json', 'get_grandmaster_top.json', 'get_master_top.json' y fusionarlos en un solo archivo 'best_players.json'
import json

with open(f"{OUTPUT_PATH}/get_challengers_top.json", "r") as f:
    challengers = json.load(f)
    tier = challengers["tier"]
    entries = challengers["entries"]
    challenger_list = [{**entry, "tier": tier} for entry in entries]

with open(f"{OUTPUT_PATH}/get_grandmaster_top.json", "r") as f:
    grandmaster = json.load(f)
    tier = grandmaster["tier"]
    entries = grandmaster["entries"]
    grandmaster_list = [{**entry, "tier": tier} for entry in entries]

with open(f"{OUTPUT_PATH}/get_master_top.json", "r") as f:
    master = json.load(f)
    tier = master["tier"]
    entries = master["entries"]
    master_list = [{**entry, "tier": tier} for entry in entries]

challenger_list.extend(grandmaster_list)
challenger_list.extend(master_list)

with open(f"{OUTPUT_PATH}/best_players.json", "w") as file:
    json.dump(challenger_list, file, indent=4)

print("Archivo Guardado 🤗")

Archivo Guardado 🤗


In [ ]:
print(len(challenger_list))

2132


## Recuperar puuid de cada jugador

"summonerId" -> "puuid"

In [11]:
import time

class Sleep:
    _perMinute = 99

    def __init__(self):
        self.listado_tiempos = []

    def count(self):
        current_t = time.time()
        self.listado_tiempos.append(current_t)
        #last_minute_t = [t for t in self.listado_tiempos if (current_t - t) <= 120]
        last_minute_t = list(filter(lambda t: (current_t - t) <= 120, self.listado_tiempos))
        #print(len(last_minute_t))
        if (len(last_minute_t) >= self._perMinute):
          sleep_t = 120 - (current_t - last_minute_t[10])
          time.sleep(sleep_t)  # Dormir
        self.listado_tiempos = last_minute_t

In [12]:
# Código consulta "summonerId" -> "puuid"
from tqdm import tqdm
import csv

def puuid_of_summonerId(players):
  """
  Consigue los puuid de los jugadores a partir de su summonerId. Guarda los
  resultados en un archivo "puuids.csv"
  """
  sleepLogic = Sleep()
  with open(f"{DATA_DIR}/puuids.csv", mode='w', newline='') as csv_file:
    fieldnames = ['puuid']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for p in tqdm(players, desc="⏳ Obteniendo Partidos"):
        try:
            sleepLogic.count()
            puuid = get_puuid_by_summoner_id(API_KEY, p["summonerId"])["puuid"]
            #print(puuid)
            writer.writerow({"puuid": puuid})
        except Exception as e:
            print(e)
            pass

In [16]:
import pandas as pd
import json

with open(f"{OUTPUT_PATH}/best_players.json", "r") as f:
    best_players = json.load(f)
    print(len(best_players))

puuid_of_summonerId(best_players)

2132


⏳ Obteniendo Partidos: 100%|██████████| 2132/2132 [43:51<00:00,  1.23s/it]


## Recuperar Partidos de cada jugador

In [23]:
# Código consulta "summonerId" -> "puuid"
from tqdm import tqdm

def match_of_puuids(puuids, cant):
  """
  Consigue los ultimos Cant partidos de jugadores a partir de su puuids .
  Guarda los resultados en un archivo "matchs_id.json"
  """
  sleepLogic = Sleep()
  json_file_path = f"{DATA_DIR}/matchs_id.json"
  with open(json_file_path, mode='w') as json_file:
    json_file.write('[')  # Comienza un nuevo array JSON
    for i, puuid in enumerate(tqdm(puuids, desc="⏳ Obteniendo PUUIDs")):
      try:
          match_ids = get_match_ids(API_KEY, puuid, "ranked", count=cant)
          sleepLogic.count()
          json.dump({"puuid": puuid, "match_ids": match_ids}, json_file)
          # Si no es el último elemento, agrega una coma
          if i < len(puuids) - 1:
            json_file.write(',\n')
      except Exception as e:
          print(e)
          pass
    json_file.write(']')  # Cierra el array JSON
    print()
  print(f"Resultados guardados en: {json_file_path} 🤗")

In [19]:
# Cargar datos de puuids.csv
df = pd.read_csv(f'{DATA_DIR}/puuids.csv')
puuids_list = df['puuid'].tolist()
print(f'{len(puuids_list)} puuids cargados 📖')

2132 puuids cargados 📖


In [24]:
# Recuperar partidos
match_of_puuids(puuids_list, 10)

⏳ Obteniendo PUUIDs: 100%|██████████| 2132/2132 [43:59<00:00,  1.24s/it]


Resultados guardados en: /content/drive/MyDrive/PPS_Esports/data/matchs_id.json 🤗


## Obtener datos de cada partido
Leer cada partido de matchs_id.json, obtener los datos de cada match_id y guardar los datos en un archivo matchs_data.json

In [34]:
import json

# Cargar datos desde el archivo JSON
json_file_path = f'{DATA_DIR}/matchs_id.json'
with open(json_file_path, 'r') as json_file:
    maths_ids = json.load(json_file)

print(f'{len(maths_ids)} maths_id cargados 📖')

2132 maths_id cargados 📖


In [35]:
# Separar listado de ids de partidos
# Nos aseguramos que no se guarde varias veces esl mismo partido
match_ids = set()
for matchs in maths_ids:
  for match_id in matchs["match_ids"]:
    match_ids.add(match_id)

In [36]:
print(f'{len(match_ids)} maths_id cargados 📖')

9968 maths_id cargados 📖


In [37]:
# Recorrer los partidos y recuperar la informacion de cada uno.
from tqdm import tqdm

sleepLogic = Sleep()
matches_list=[]
for match_id in tqdm(match_ids):
  sleepLogic.count()
  matches_list.append(get_match_details(API_KEY, match_id))

 93%|█████████▎| 9316/9968 [3:17:42<13:50,  1.27s/it]


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [38]:
# Guardar resultados en JSON
with open(f"{OUTPUT_PATH}/matches_details.json", "w") as file:
    json.dump(matches_list, file, indent=4)

In [ ]:
import json

# Cargar datos desde el archivo JSON
json_file_path = f'{DATA_DIR}/matches_details.json'
with open(json_file_path, 'r') as json_file:
    maths_details = json.load(json_file)

In [40]:
print(f'{len(maths_details)}')

9316
